In [1]:
from pprint import pprint
import jmespath
import requests
from bs4 import BeautifulSoup as BS4
from textwrap import dedent
import re
from modules.shopping import ShoppingList

In [44]:
string = '100 €'

In [45]:
test = re.findall(r'(\d+.\d+)',string)
float(test[0].replace(',','.'))

100.0

'1,99 €'